**PRÁTICA PYDANTIC**

In [2]:
from datetime import datetime 
from typing import Optional
from uuid import UUID, uuid4

from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, Field, EmailStr

In [3]:
app = FastAPI() #Instancia do FastAPI

In [4]:
# Banco de dados em memória
USERS = []
TASKS = []

In [5]:
#Nossa classe de usuário
class User(BaseModel):
    id: UUID = Field(default_factory=uuid4, description="Unique identifier") #Campo de id usando como identificador o uuid
    name: str = Field(..., description="User's name") #Campo do nome usando como identificador o tipo str(String)
    email: EmailStr = Field(..., description="User's email") ##Campo do email usando como identificador o tipo EmailStr

In [6]:
#Nossa classe de tarefas
class Task(BaseModel):
    id: UUID = Field(default_factory=uuid4, description="Task ID")
    title: str = Field(..., description="Task title")
    description: Optional[str] = Field(None, description="Task description")
    created_at: datetime = Field(default_factory=datetime.now, description="Creation timestamp")
    completed: bool = Field(default=False, description="Task completion status")
    user_id: Optional[UUID] = Field(None, description="User assigned to task")

In [7]:
#Usamos o método post para fazer uma requisição de criação de usuário
@app.post("/users", response_model=User)
async def create_user(user: User): #Nossa função assincrona
    USERS.append(user) #Adiciona o novo usuario a lista de usuarios
    return user #retorna o novo usuario

In [8]:
#Aqui usamos o método get para fazer a requisição para recuperar os dados de todos usuários existentes
@app.get("/users", response_model=list[User])
async def get_users(): #Nossa função assincrona
    return USERS #retorna a lista de usuarios

In [9]:
#Usamos o método post para fazer uma requisição de criação de tarefas
@app.post("/tasks", response_model=Task)
async def create_task(task: Task): #Nossa função assincrona
    TASKS.append(task) #Adiciona a nova tarefa a lista de tarefas
    return task #

In [10]:
#Metodo get para fazer a requisição das tarefas já existentes
@app.get("/tasks", response_model=list[Task])
async def get_tasks():
    return TASKS

In [11]:
@app.put("/tasks/{task_id}/complete", response_model=Task)  # Define o endpoint PUT para marcar uma tarefa como completa
async def complete_task(task_id: UUID):  # Função assíncrona que recebe o 'task_id' como parâmetro, sendo um UUID
    for task in TASKS:  # Itera sobre a lista de tarefas armazenadas em 'TASKS'
        if task.id == task_id:  # Verifica se o 'id' da tarefa atual é igual ao 'task_id' fornecido na URL
            task.completed = True  # Marca a tarefa como completa, alterando o atributo 'completed' para True
            return task  # Retorna a tarefa com a atualização, como resposta, formatada de acordo com o modelo 'Task'
    
    # Se nenhuma tarefa for encontrada com o 'task_id', lança uma exceção HTTP 404 (não encontrada)
    raise HTTPException(status_code=404, detail="Task not found")  # Retorna erro 404 se a tarefa não for encontrada


In [12]:
@app.get("/users/{user_id}/tasks", response_model=list[Task])  # Define o endpoint GET para retornar as tarefas de um usuário específico
async def get_user_tasks(user_id: UUID):  # Função assíncrona que recebe o 'user_id' como parâmetro, sendo um UUID
    user_tasks = [task for task in TASKS if task.user_id == user_id]  # Filtra todas as tarefas que pertencem ao 'user_id' fornecido
    return user_tasks  # Retorna a lista de tarefas do usuário, formatada de acordo com o modelo 'Task'


In [13]:
# Testes automáticos
def main():
    with TestClient(app) as client:
        # Criar usuários
        response = client.post("/users", json={"name": "Alice", "email": "alice@example.com"})
        assert response.status_code == 200
        user_id = response.json()["id"]

        # Criar uma tarefa
        response = client.post("/tasks", json={"title": "Buy groceries", "description": "Milk, Eggs, Bread", "user_id": user_id})
        assert response.status_code == 200
        task_id = response.json()["id"]
        
        # Marcar tarefa como concluída
        response = client.put(f"/tasks/{task_id}/complete")
        assert response.status_code == 200
        assert response.json()["completed"] is True
        
        # Buscar tarefas do usuário
        response = client.get(f"/users/{user_id}/tasks")
        assert response.status_code == 200
        assert len(response.json()) == 1
        
        print(" Todos os testes passaram!")

In [14]:
if __name__ == "__main__":
    main()

 Todos os testes passaram!
